In [1]:
! pip install -q langchain-core langchain-community langchain-chroma langchain-google-genai pypdf

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.merger_retriever import MergerRetriever
import chromadb

from google.colab import userdata
import os

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
document_harry = PyPDFLoader(file_path="/content/harry_potter_book.pdf").load()

In [6]:
document_summaries = PyPDFLoader(file_path="/content/lightning_thief.pdf").load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [8]:
doc_harry = text_splitter.split_documents(document_harry)
doc_summaries = text_splitter.split_documents(document_summaries)

In [9]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [10]:
client_settings = chromadb.config.Settings(
    is_persistent=True,
    anonymized_telemetry=False,
)

In [11]:
harry_vector = Chroma.from_documents(documents=doc_harry,embedding=embedding,collection_name="harry",persist_directory="harry_vector",collection_metadata={"hnsw":"cosine"},client_settings=client_settings)
summaries_vector = Chroma.from_documents(documents=doc_summaries,embedding=embedding,collection_name="summary",persist_directory="summary_vector",collection_metadata={"hnsw":"cosine"},client_settings=client_settings)

In [12]:
harry_retriever = harry_vector.as_retriever(search_type="mmr",search_kwargs={"k":2,"include_metadata":True})
summaries_retriever = summaries_vector.as_retriever(search_type="mmr",search_kwargs={"k":2,"include_metadata":True})

In [13]:
lotr = MergerRetriever(retrievers=[harry_retriever,summaries_retriever])

In [14]:
for chunks in lotr.get_relevant_documents("Who is harry potter"):
  print(chunks.page_content)

<ipython-input-14-5c4fbf03a3bc>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  for chunks in lotr.get_relevant_documents("Who is harry potter"):


© OCT 2017 | IRE Journals | Volume 1 Issue 4 | ISSN: 2456-8880 
 
IRE 1700079          ICONIC RESEARCH AND ENGINEERING JOURNALS 71 
support system of friends that Voldemort can never 
hope to match. 
 
Harry's first contact with the Wizarding world is 
through a half giant, Rebus Hagrid, keeper of 
grounds and keys at Hogwarts. Hagrid reveals some 
of Harry's history. Harry survived with only a 
lightning shaped scar on his forehead as a memento 
of the attack and Voldemort reign of terror, Harry has 
become a living legend in the Wizarding world. 
Wizards represent all that the true Muggles most 
fears: The are plainly out casts and comfortable with 
being so not hing is more unnerving to the truly 
conventional than the unashamed misfit. Harry's 
eleventh birthday a series of very bizarre events lead 
to the discovery of Harry's true identity, he's a wizard 
 
IV. CONCLUSION 
J. K. Rowling transformed the lives of a generation 
with her magical Harry Potter book but now the author
wa

In [15]:
for chunks in lotr.get_relevant_documents("Who wrote percy Jackson"):
  print(chunks.page_content)

and Lumos. The novels chronicle the life of a young wizard 
Harry Potter, and his friends Hermione Granger and Ron 
Weasley, all of whom are students at Hogwarts  School of 
witchcraft and wizardry. When the first novel of the series, 
Harry Potter and the Philosopher’s Stone, the main story 
are concerns Harry’s struggle against Lord Voldemort a 
dark wizard who intends to become immortal overthrow the 
wizard governing body known as the Ministry of magic and 
reference term that me ans non -magical people. The 
environment Rowling created is intimately connected to 
reality. 
 
Index Terms: CEMP, Environmental Risk, Impact factors, 
Risk Assessment. 
 
I. INTRODUCTION 
 
Joanne Rowling, pen names J. K. Rowling and 
Robert Galbrath is a British Novelist best known as 
the author of the Harry Potter fantasy series. she was 
born 31 July 1965, in Yate Gloucestershire England. 
Rowling was working as a researcher and bilingual 
secretary for Amnesty International when she
About the Autho

In [16]:
from langchain_community.document_transformers import EmbeddingsRedundantFilter, LongContextReorder
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline


In [17]:
from re import search
from itertools import compress
filter = EmbeddingsRedundantFilter(embeddings=embedding)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(
    transformers=[filter, reordering]
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=lotr,search_type="mmr",search_kwargs={"k":2,"include_metadata":True}
)


In [18]:
from langchain.chains import RetrievalQA

In [19]:
qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff",retriever=compression_retriever,return_source_documents=True)

In [20]:
query = "who is harry potter?"
results = qa(query)
print(results["result"])

<ipython-input-20-a2da0259f3df>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = qa(query)


Harry Potter is an orphan who escapes his miserable childhood with the Dursley family. He is also a wizard.


In [21]:
query = "Author of harry potter"
results = qa(query)
print(results["result"])

The author of the Harry Potter series is J.K. Rowling, also known as Joanne Rowling and Robert Galbrath.


In [22]:
query = "Author of Percy Jackson"
results = qa(query)
print(results["result"])

Rick Riordan is the author of the Percy Jackson series.
